# pyrealsense를 활용해 point cloud와 color array를 저장합니다
## <사전 param 설정>

In [1]:
# License: Apache 2.0. See LICENSE file in root directory.
# Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

"""
OpenCV and Numpy Point cloud Software Renderer
This sample is mostly for demonstration and educational purposes.
It really doesn't offer the quality or performance that can be
achieved with hardware acceleration.
Usage:
------
Mouse: 
    Drag with left button to rotate around pivot (thick small axes), 
    with right button to translate and the wheel to zoom.
Keyboard: 
    [p]     Pause
    [r]     Reset View
    [d]     Cycle through decimation values
    [z]     Toggle point scaling
    [c]     Toggle color source
    [s]     Save PNG (./out.png)
    [e]     Export points to ply (./out.ply)
    [q\ESC] Quit
"""

import math
import time
import cv2
import numpy as np
import pyrealsense2 as rs
import os


class AppState:

    def __init__(self, *args, **kwargs):
        self.WIN_NAME = 'RealSense'
        self.pitch, self.yaw = math.radians(-10), math.radians(-15)
        self.translation = np.array([0, 0, -1], dtype=np.float32)
        self.distance = 2
        self.prev_mouse = 0, 0
        self.mouse_btns = [False, False, False]
        self.paused = False
        self.decimate = 1
        self.scale = True
        self.color = True

    def reset(self):
        self.pitch, self.yaw, self.distance = 0, 0, 2
        self.translation[:] = 0, 0, -1

    @property
    def rotation(self):
        Rx, _ = cv2.Rodrigues((self.pitch, 0, 0))
        Ry, _ = cv2.Rodrigues((0, self.yaw, 0))
        return np.dot(Ry, Rx).astype(np.float32)

    @property
    def pivot(self):
        return self.translation + np.array((0, 0, self.distance), dtype=np.float32)


state = AppState()

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get stream profile and camera intrinsics
profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
w, h = depth_intrinsics.width, depth_intrinsics.height

# Processing blocks
pc = rs.pointcloud()
decimate = rs.decimation_filter()
decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)
colorizer = rs.colorizer()


def mouse_cb(event, x, y, flags, param):

    if event == cv2.EVENT_LBUTTONDOWN:
        state.mouse_btns[0] = True

    if event == cv2.EVENT_LBUTTONUP:
        state.mouse_btns[0] = False

    if event == cv2.EVENT_RBUTTONDOWN:
        state.mouse_btns[1] = True

    if event == cv2.EVENT_RBUTTONUP:
        state.mouse_btns[1] = False

    if event == cv2.EVENT_MBUTTONDOWN:
        state.mouse_btns[2] = True

    if event == cv2.EVENT_MBUTTONUP:
        state.mouse_btns[2] = False

    if event == cv2.EVENT_MOUSEMOVE:

        h, w = out.shape[:2]
        dx, dy = x - state.prev_mouse[0], y - state.prev_mouse[1]

        if state.mouse_btns[0]:
            state.yaw += float(dx) / w * 2
            state.pitch -= float(dy) / h * 2

        elif state.mouse_btns[1]:
            dp = np.array((dx / w, dy / h, 0), dtype=np.float32)
            state.translation -= np.dot(state.rotation, dp)

        elif state.mouse_btns[2]:
            dz = math.sqrt(dx**2 + dy**2) * math.copysign(0.01, -dy)
            state.translation[2] += dz
            state.distance -= dz

    if event == cv2.EVENT_MOUSEWHEEL:
        dz = math.copysign(0.1, flags)
        state.translation[2] += dz
        state.distance -= dz

    state.prev_mouse = (x, y)


cv2.namedWindow(state.WIN_NAME, cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow(state.WIN_NAME, w, h)
cv2.setMouseCallback(state.WIN_NAME, mouse_cb)


def project(v):
    """project 3d vector array to 2d"""
    h, w = out.shape[:2]
    view_aspect = float(h)/w

    # ignore divide by zero for invalid depth
    with np.errstate(divide='ignore', invalid='ignore'):
        proj = v[:, :-1] / v[:, -1, np.newaxis] * \
            (w*view_aspect, h) + (w/2.0, h/2.0)

    # near clipping
    znear = 0.03
    proj[v[:, 2] < znear] = np.nan
    return proj


def view(v):
    """apply view transformation on vector array"""
    return np.dot(v - state.pivot, state.rotation) + state.pivot - state.translation


def line3d(out, pt1, pt2, color=(0x80, 0x80, 0x80), thickness=1):
    """draw a 3d line from pt1 to pt2"""
    p0 = project(pt1.reshape(-1, 3))[0]
    p1 = project(pt2.reshape(-1, 3))[0]
    if np.isnan(p0).any() or np.isnan(p1).any():
        return
    p0 = tuple(p0.astype(int))
    p1 = tuple(p1.astype(int))
    rect = (0, 0, out.shape[1], out.shape[0])
    inside, p0, p1 = cv2.clipLine(rect, p0, p1)
    if inside:
        cv2.line(out, p0, p1, color, thickness, cv2.LINE_AA)


def grid(out, pos, rotation=np.eye(3), size=1, n=10, color=(0x80, 0x80, 0x80)):
    """draw a grid on xz plane"""
    pos = np.array(pos)
    s = size / float(n)
    s2 = 0.5 * size
    for i in range(0, n+1):
        x = -s2 + i*s
        line3d(out, view(pos + np.dot((x, 0, -s2), rotation)),
               view(pos + np.dot((x, 0, s2), rotation)), color)
    for i in range(0, n+1):
        z = -s2 + i*s
        line3d(out, view(pos + np.dot((-s2, 0, z), rotation)),
               view(pos + np.dot((s2, 0, z), rotation)), color)


def axes(out, pos, rotation=np.eye(3), size=0.075, thickness=2):
    """draw 3d axes"""
    line3d(out, pos, pos +
           np.dot((0, 0, size), rotation), (0xff, 0, 0), thickness)
    line3d(out, pos, pos +
           np.dot((0, size, 0), rotation), (0, 0xff, 0), thickness)
    line3d(out, pos, pos +
           np.dot((size, 0, 0), rotation), (0, 0, 0xff), thickness)


def frustum(out, intrinsics, color=(0x40, 0x40, 0x40)):
    """draw camera's frustum"""
    orig = view([0, 0, 0])
    w, h = intrinsics.width, intrinsics.height

    for d in range(1, 6, 2):
        def get_point(x, y):
            p = rs.rs2_deproject_pixel_to_point(intrinsics, [x, y], d)
            line3d(out, orig, view(p), color)
            return p

        top_left = get_point(0, 0)
        top_right = get_point(w, 0)
        bottom_right = get_point(w, h)
        bottom_left = get_point(0, h)

        line3d(out, view(top_left), view(top_right), color)
        line3d(out, view(top_right), view(bottom_right), color)
        line3d(out, view(bottom_right), view(bottom_left), color)
        line3d(out, view(bottom_left), view(top_left), color)


def pointcloud(out, verts, texcoords, color, painter=True):
    """draw point cloud with optional painter's algorithm"""
    if painter:
        # Painter's algo, sort points from back to front

        # get reverse sorted indices by z (in view-space)
        # https://gist.github.com/stevenvo/e3dad127598842459b68
        v = view(verts)
        s = v[:, 2].argsort()[::-1]
        proj = project(v[s])
    else:
        proj = project(view(verts))

    if state.scale:
        proj *= 0.5**state.decimate

    h, w = out.shape[:2]

    # proj now contains 2d image coordinates
    j, i = proj.astype(np.uint32).T

    # create a mask to ignore out-of-bound indices
    im = (i >= 0) & (i < h)
    jm = (j >= 0) & (j < w)
    m = im & jm

    cw, ch = color.shape[:2][::-1]
    if painter:
        # sort texcoord with same indices as above
        # texcoords are [0..1] and relative to top-left pixel corner,
        # multiply by size and add 0.5 to center
        v, u = (texcoords[s] * (cw, ch) + 0.5).astype(np.uint32).T
    else:
        v, u = (texcoords * (cw, ch) + 0.5).astype(np.uint32).T
    # clip texcoords to image
    np.clip(u, 0, ch-1, out=u)
    np.clip(v, 0, cw-1, out=v)

    # perform uv-mapping
    out[i[m], j[m]] = color[u[m], v[m]]



out = np.empty((h, w, 3), dtype=np.uint8)

def get_verts(name):
    npsavePath='C://Users//user//Desktop//openPose//pointcloud/'+name
    pc_token = np.load(npsavePath)
    return pc_token

def zero_filter(x):
    if (x[2] > 1.5) and (x[2] < 3):
        return [x[0],x[1],x[2]]
    else:
        return [0,0,0]
    
def binary(x):
    if (x == 0):
        return 0
    else:
        return 0.5
   

In [2]:
framecount =0

# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2

# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)


Depth Scale is:  0.0010000000474974513


## <데이터 저장>

In [3]:
i=0
sums=np.zeros((76800, 3))
u_sum=np.zeros((76800, 3))
framecount =0

while True:
    # Grab camera data
    if not state.paused:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()

        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_frame = decimate.process(depth_frame)

        # Grab new intrinsics (may be changed by decimation)
        depth_intrinsics = rs.video_stream_profile(
            depth_frame.profile).get_intrinsics()
        w, h = depth_intrinsics.width, depth_intrinsics.height

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        depth_colormap = np.asanyarray(
            colorizer.colorize(depth_frame).get_data())

        if state.color:
            mapped_frame, color_source = color_frame, color_image
        else:
            mapped_frame, color_source = depth_frame, depth_colormap
        
        points = pc.calculate(depth_frame)
        pc.map_to(mapped_frame)
        
        # Pointcloud data to arrays
        v, t = points.get_vertices(), points.get_texture_coordinates()
        verts = np.asanyarray(v).view(np.float32).reshape(-1, 3)  # xyz
        npsavePath='C://Users//user//Desktop//openPose//outputs//pointcloud/'+str(framecount).zfill(5)
        np.save(npsavePath,verts)
        
        texcoords = np.asanyarray(t).view(np.float32).reshape(-1, 2)  # uv
        tcsavePath='C://Users//user//Desktop//openPose//outputs//texcoords/'+str(framecount)
        #np.save(tcsavePath,texcoords)
        
        plysavePath='C://Users//user//Desktop//openPose//outputs//ply/'+str(framecount)+'.ply'
        #points.export_to_ply(plysavePath, mapped_frame)
        
        ###############################################
        
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            print('Error, fail to get depth or color frame')
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        savePath_d = 'C://Users//user//Desktop//openPose//outputs//test_depth_array/' + str(framecount).zfill(5)
        savePath_c = 'C://Users//user//Desktop//openPose//outputs//test_color_array/' + str(framecount).zfill(5)
        np.save(savePath_d,depth_image)
        np.save(savePath_c,color_image)#1개의 배열을 NumPy format의 바이너리 파일로 저장하기 (Save a single array to a binary file in NumPy format)
        
        # Render images
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((color_image, depth_colormap))
        cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Example', images)
        
        key = cv2.waitKey(1)
        print('frame : ',framecount)
        print('time : 'framecount/30,'sec')
        framecount = framecount+1
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break

# Stop streaming

frame :  0
frame :  1
frame :  2
frame :  3
frame :  4
frame :  5
frame :  6
frame :  7
frame :  8
frame :  9
frame :  10
frame :  11
frame :  12
frame :  13
frame :  14
frame :  15
frame :  16
frame :  17
frame :  18
frame :  19
frame :  20
frame :  21
frame :  22
frame :  23
frame :  24
frame :  25
frame :  26
frame :  27
frame :  28
frame :  29
frame :  30
frame :  31
frame :  32
frame :  33
frame :  34
frame :  35
frame :  36
frame :  37
frame :  38
frame :  39
frame :  40
frame :  41
frame :  42
frame :  43
frame :  44
frame :  45
frame :  46
frame :  47
frame :  48
frame :  49
frame :  50
frame :  51
frame :  52
frame :  53
frame :  54
frame :  55
frame :  56
frame :  57
frame :  58
frame :  59
frame :  60
frame :  61
frame :  62
frame :  63
frame :  64
frame :  65
frame :  66
frame :  67
frame :  68
frame :  69
frame :  70
frame :  71
frame :  72
frame :  73
frame :  74
frame :  75
frame :  76
frame :  77
frame :  78
frame :  79
frame :  80
frame :  81
frame :  82
frame :  83
fr

frame :  644
frame :  645
frame :  646
frame :  647
frame :  648
frame :  649
frame :  650
frame :  651
frame :  652
frame :  653
frame :  654
frame :  655
frame :  656
frame :  657
frame :  658
frame :  659
frame :  660
frame :  661
frame :  662
frame :  663
frame :  664
frame :  665
frame :  666
frame :  667
frame :  668
frame :  669
frame :  670
frame :  671
frame :  672
frame :  673
frame :  674
frame :  675
frame :  676
frame :  677
frame :  678
frame :  679
frame :  680
frame :  681
frame :  682
frame :  683
frame :  684
frame :  685
frame :  686
frame :  687
frame :  688
frame :  689
frame :  690
frame :  691
frame :  692
frame :  693
frame :  694
frame :  695
frame :  696
frame :  697
frame :  698
frame :  699
frame :  700
frame :  701
frame :  702
frame :  703
frame :  704
frame :  705
frame :  706
frame :  707
frame :  708
frame :  709
frame :  710
frame :  711
frame :  712
frame :  713
frame :  714
frame :  715
frame :  716
frame :  717
frame :  718
frame :  719
frame :  720

# 저장한 array를 color image와 video로 변환합니다

In [5]:
import os
import numpy as np
import cv2

videopath = 'C://Users//user//Desktop//openPose//outputs//test_color_array/'
frame_list = sorted(os.listdir(videopath)) #피험자 리스트
print(frame_list)

outputVideo = []

for j in frame_list :#frame별 slice
    open_frame = videopath +'//' + j
    save_load = np.load(open_frame)
    imagepath = 'C://Users//user//Desktop//openPose//outputs//frames/'
    imagename = j+'.jpg'
    cv2.imwrite(os.path.join(imagepath ,imagename), save_load)
    print(j,save_load.shape)
    outputVideo.append(save_load)


out = cv2.VideoWriter('C://Users/user/Desktop/openPose//outputs/raw_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, (640,480))

for i in range(len(outputVideo)):
    out.write(outputVideo[i])
              
out.release()

['00000.npy', '00001.npy', '00002.npy', '00003.npy', '00004.npy', '00005.npy', '00006.npy', '00007.npy', '00008.npy', '00009.npy', '00010.npy', '00011.npy', '00012.npy', '00013.npy', '00014.npy', '00015.npy', '00016.npy', '00017.npy', '00018.npy', '00019.npy', '00020.npy', '00021.npy', '00022.npy', '00023.npy', '00024.npy', '00025.npy', '00026.npy', '00027.npy', '00028.npy', '00029.npy', '00030.npy', '00031.npy', '00032.npy', '00033.npy', '00034.npy', '00035.npy', '00036.npy', '00037.npy', '00038.npy', '00039.npy', '00040.npy', '00041.npy', '00042.npy', '00043.npy', '00044.npy', '00045.npy', '00046.npy', '00047.npy', '00048.npy', '00049.npy', '00050.npy', '00051.npy', '00052.npy', '00053.npy', '00054.npy', '00055.npy', '00056.npy', '00057.npy', '00058.npy', '00059.npy', '00060.npy', '00061.npy', '00062.npy', '00063.npy', '00064.npy', '00065.npy', '00066.npy', '00067.npy', '00068.npy', '00069.npy', '00070.npy', '00071.npy', '00072.npy', '00073.npy', '00074.npy', '00075.npy', '00076.npy'

00016.npy (480, 640, 3)
00017.npy (480, 640, 3)
00018.npy (480, 640, 3)
00019.npy (480, 640, 3)
00020.npy (480, 640, 3)
00021.npy (480, 640, 3)
00022.npy (480, 640, 3)
00023.npy (480, 640, 3)
00024.npy (480, 640, 3)
00025.npy (480, 640, 3)
00026.npy (480, 640, 3)
00027.npy (480, 640, 3)
00028.npy (480, 640, 3)
00029.npy (480, 640, 3)
00030.npy (480, 640, 3)
00031.npy (480, 640, 3)
00032.npy (480, 640, 3)
00033.npy (480, 640, 3)
00034.npy (480, 640, 3)
00035.npy (480, 640, 3)
00036.npy (480, 640, 3)
00037.npy (480, 640, 3)
00038.npy (480, 640, 3)
00039.npy (480, 640, 3)
00040.npy (480, 640, 3)
00041.npy (480, 640, 3)
00042.npy (480, 640, 3)
00043.npy (480, 640, 3)
00044.npy (480, 640, 3)
00045.npy (480, 640, 3)
00046.npy (480, 640, 3)
00047.npy (480, 640, 3)
00048.npy (480, 640, 3)
00049.npy (480, 640, 3)
00050.npy (480, 640, 3)
00051.npy (480, 640, 3)
00052.npy (480, 640, 3)
00053.npy (480, 640, 3)
00054.npy (480, 640, 3)
00055.npy (480, 640, 3)
00056.npy (480, 640, 3)
00057.npy (480, 

00367.npy (480, 640, 3)
00368.npy (480, 640, 3)
00369.npy (480, 640, 3)
00370.npy (480, 640, 3)
00371.npy (480, 640, 3)
00372.npy (480, 640, 3)
00373.npy (480, 640, 3)
00374.npy (480, 640, 3)
00375.npy (480, 640, 3)
00376.npy (480, 640, 3)
00377.npy (480, 640, 3)
00378.npy (480, 640, 3)
00379.npy (480, 640, 3)
00380.npy (480, 640, 3)
00381.npy (480, 640, 3)
00382.npy (480, 640, 3)
00383.npy (480, 640, 3)
00384.npy (480, 640, 3)
00385.npy (480, 640, 3)
00386.npy (480, 640, 3)
00387.npy (480, 640, 3)
00388.npy (480, 640, 3)
00389.npy (480, 640, 3)
00390.npy (480, 640, 3)
00391.npy (480, 640, 3)
00392.npy (480, 640, 3)
00393.npy (480, 640, 3)
00394.npy (480, 640, 3)
00395.npy (480, 640, 3)
00396.npy (480, 640, 3)
00397.npy (480, 640, 3)
00398.npy (480, 640, 3)
00399.npy (480, 640, 3)
00400.npy (480, 640, 3)
00401.npy (480, 640, 3)
00402.npy (480, 640, 3)
00403.npy (480, 640, 3)
00404.npy (480, 640, 3)
00405.npy (480, 640, 3)
00406.npy (480, 640, 3)
00407.npy (480, 640, 3)
00408.npy (480, 

00721.npy (480, 640, 3)
00722.npy (480, 640, 3)
00723.npy (480, 640, 3)
00724.npy (480, 640, 3)
00725.npy (480, 640, 3)
00726.npy (480, 640, 3)
00727.npy (480, 640, 3)
00728.npy (480, 640, 3)
00729.npy (480, 640, 3)
00730.npy (480, 640, 3)
00731.npy (480, 640, 3)
00732.npy (480, 640, 3)
00733.npy (480, 640, 3)
00734.npy (480, 640, 3)
00735.npy (480, 640, 3)
00736.npy (480, 640, 3)
00737.npy (480, 640, 3)
00738.npy (480, 640, 3)
00739.npy (480, 640, 3)
00740.npy (480, 640, 3)
00741.npy (480, 640, 3)
00742.npy (480, 640, 3)
00743.npy (480, 640, 3)
00744.npy (480, 640, 3)
00745.npy (480, 640, 3)
00746.npy (480, 640, 3)
00747.npy (480, 640, 3)
00748.npy (480, 640, 3)
00749.npy (480, 640, 3)
00750.npy (480, 640, 3)
00751.npy (480, 640, 3)
00752.npy (480, 640, 3)
00753.npy (480, 640, 3)
00754.npy (480, 640, 3)
00755.npy (480, 640, 3)
00756.npy (480, 640, 3)
00757.npy (480, 640, 3)
00758.npy (480, 640, 3)
00759.npy (480, 640, 3)
00760.npy (480, 640, 3)
00761.npy (480, 640, 3)
00762.npy (480, 

# 저장한 pointcloud와 color array를 matching 합니다

In [93]:
def joint_pixelToPoint(path_PC, path_json):
    
    path_depth= path_PC
    path_json = path_json
    json_list = os.listdir(path_json) #path에 있는 json 리스트
    depth_list = os.listdir(path_depth)#path에 있는 depth array 리스트
    joint = {'r_shoulder' : ['1','2','3'], 'r_elbow' : ['2','3','4'], 'l_shoulder' : ['1','5','6'], 'l_elbow' : ['5','6','7'],'r_wrist':['0','4'],'l_wrist':['0','7']}
    matched = {}
    
    frame = 0
    matched = {}
    for j,k in zip(json_list, depth_list) :
        indicate=[]

        open_json = path_json +'//' + j
        open_depth = path_depth +'//' + k
        nf_depthData = np.load(open_depth)
        nf_depthData=nf_depthData.reshape(240,320,3)
        nf_depthData=np.flip(nf_depthData,axis=0)
        nf_depthData=np.flip(nf_depthData,axis=1)
        print('frame :',frame)
        print('json : ',j,'   /    Pointcloud : ',k)


        with open(open_json, encoding="utf-8") as data_file :
            data = json.load(data_file, object_pairs_hook=OrderedDict)

        for people, part_candidates in enumerate(data["part_candidates"]) :
            if people > 0 : print(", ", end="")
            for njoint in joint:

                try:
                    x = part_candidates[joint[njoint][1]][0]
                    pixel_x = int(x/2)
                except:
                    print("cannnot define",njoint, 'x')
                    continue

                try:
                    y = part_candidates[joint[njoint][1]][1]
                    pixel_y = int(y/2)
                except :
                    print("cannnot define",njoint, 'y')
                    continue
                print(njoint,' x: ',pixel_x,'y: ',pixel_y)
                    
                PC_point=nf_depthData[pixel_y][pixel_x]
                print(PC_point)
                
                indicate.append([njoint,PC_point])
                print()

            matched[frame]=[indicate]
        print('================frame end ==================')
        frame = frame + 1


    print("complete")
  
    return matched

In [94]:
# PCpath = 'C://Users//user//Desktop//openPose//outputs//pointcloud/'
# frame_list = sorted(os.listdir(PCpath)) 
# PCs=[]
# framecount=0
# for j in frame_list :#frame별 slice
#     open_frame = PCpath +'//' + j
#     save_load = np.load(open_frame)
#     npsavePath='C://Users//user//Desktop//openPose//outputs//pointcloud/'+str(framecount).zfill(5)
#     np.save(npsavePath,save_load)
#     framecount+=1

In [95]:
import os
import json
from collections import OrderedDict
import numpy as np
import math

path_PC = 'C://Users//user//Desktop//openPose//outputs//pointcloud/'
path_json = 'C://Users//user//Desktop//openPose//outputs//json/'

matched = joint_pixelToPoint(path_PC, path_json)
print(matched)

frame : 0
json :  raw_video_000000000000_keypoints.json    /    Pointcloud :  00000.npy
r_shoulder  x:  175 y:  137
[-0.04412615 -0.04855199  0.551     ]

cannnot define r_elbow x
l_shoulder  x:  198 y:  145
[-0.11955939 -0.07792321  0.60300004]

cannnot define l_elbow x
cannnot define r_wrist x
cannnot define l_wrist x
================frame end ==================
frame : 1
json :  raw_video_000000000001_keypoints.json    /    Pointcloud :  00001.npy
r_shoulder  x:  139 y:  127
[ 0.03483016 -0.01219401  0.33200002]

cannnot define r_elbow x
l_shoulder  x:  173 y:  133
[-0.03790483 -0.03668577  0.54300004]

l_elbow  x:  179 y:  157
[-0.05595508 -0.10613513  0.55600005]

cannnot define r_wrist x
cannnot define l_wrist x
================frame end ==================
frame : 2
json :  raw_video_000000000002_keypoints.json    /    Pointcloud :  00002.npy
r_shoulder  x:  133 y:  132
[ 0. -0.  0.]

cannnot define r_elbow x
l_shoulder  x:  164 y:  132
[-0. -0.  0.]

l_elbow  x:  169 y:  158
[-0

frame : 71
json :  raw_video_000000000071_keypoints.json    /    Pointcloud :  00071.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
cannnot define r_wrist x
cannnot define l_wrist x
================frame end ==================
frame : 72
json :  raw_video_000000000072_keypoints.json    /    Pointcloud :  00072.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
cannnot define r_wrist x
cannnot define l_wrist x
================frame end ==================
frame : 73
json :  raw_video_000000000073_keypoints.json    /    Pointcloud :  00073.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
cannnot define r_wrist x
cannnot define l_wrist x
================frame end ==================
frame : 74
json :  raw_video_000000000074_keypoints.json    /    Pointcloud :  00074.npy
cannnot define r_shoulder x
cannnot defin

cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
cannnot define r_wrist x
cannnot define l_wrist x
================frame end ==================
frame : 148
json :  raw_video_000000000148_keypoints.json    /    Pointcloud :  00148.npy
r_shoulder  x:  258 y:  167
[-1.8576927 -0.8884323  3.667    ]

cannnot define r_elbow x
l_shoulder  x:  177 y:  160
[-0.09298167 -0.21228957  1.029     ]

cannnot define l_elbow x
cannnot define r_wrist x
cannnot define l_wrist x
================frame end ==================
frame : 149
json :  raw_video_000000000149_keypoints.json    /    Pointcloud :  00149.npy
r_shoulder  x:  240 y:  163
[-0. -0.  0.]

cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
cannnot define r_wrist x
cannnot define l_wrist x
================frame end ==================
frame : 150
json :  raw_video_000000000150_keypoints.json    /    Pointcloud :  00150.npy
r_shoulder  x:  220 y:  159
[-0. -0. 

l_wrist  x:  288 y:  136
[-1.439794  -0.1808078  2.1790001]

================frame end ==================
frame : 190
json :  raw_video_000000000190_keypoints.json    /    Pointcloud :  00190.npy
r_shoulder  x:  169 y:  39
[-0.03285071  0.27712405  0.66700006]

r_elbow  x:  153 y:  97
[0. 0. 0.]

l_shoulder  x:  247 y:  33
[-1.1935897  1.1836158  2.6520002]

l_elbow  x:  268 y:  90
[-0.  0.  0.]

r_wrist  x:  139 y:  137
[ 0. -0.  0.]

l_wrist  x:  286 y:  137
[-1.4226031  -0.19270998  2.187     ]

================frame end ==================
frame : 191
json :  raw_video_000000000191_keypoints.json    /    Pointcloud :  00191.npy
r_shoulder  x:  169 y:  39
[-0.12037051  1.0154289   2.444     ]

r_elbow  x:  153 y:  97
[0.04836415 0.17227365 1.4670001 ]

l_shoulder  x:  247 y:  33
[-1.2093421  1.1992366  2.687    ]

l_elbow  x:  268 y:  90
[-0.  0.  0.]

r_wrist  x:  141 y:  137
[ 0.13343209 -0.12424375  1.4100001 ]

l_wrist  x:  284 y:  137
[-1.4148511  -0.19473666  2.21      ]

=====

[-0.  0.  0.]

r_wrist  x:  138 y:  145
[ 0.15957081 -0.18737753  1.45      ]

l_wrist  x:  266 y:  138
[-0. -0.  0.]

================frame end ==================
frame : 229
json :  raw_video_000000000229_keypoints.json    /    Pointcloud :  00229.npy
r_shoulder  x:  163 y:  41
[-0.02974689  0.6543994   1.6150001 ]

r_elbow  x:  150 y:  100
[0. 0. 0.]

l_shoulder  x:  236 y:  35
[-1.4096804  1.5618709  3.5820003]

l_elbow  x:  253 y:  92
[-0.  0.  0.]

r_wrist  x:  138 y:  145
[ 0.15957081 -0.18737753  1.45      ]

l_wrist  x:  266 y:  138
[-1.477714  -0.2516016  2.6980002]

================frame end ==================
frame : 230
json :  raw_video_000000000230_keypoints.json    /    Pointcloud :  00230.npy
r_shoulder  x:  162 y:  42
[-0.0215541  0.6493009  1.623    ]

r_elbow  x:  149 y:  100
[0. 0. 0.]

l_shoulder  x:  235 y:  35
[-1.3229139  1.485129   3.4060001]

l_elbow  x:  252 y:  92
[-1.0742776   0.32317898  2.2580001 ]

r_wrist  x:  137 y:  146
[ 0.16656122 -0.19429123  1.44

...

json :  raw_video_000000000926_keypoints.json    /    Pointcloud :  00926.npy
r_shoulder  x:  119 y:  58
[0.55555606 0.8502291  2.6750002 ]

r_elbow  x:  96 y:  92
[0.8948528  0.39302456 2.746     ]

l_shoulder  x:  181 y:  51
[-0.3072375  0.980064   2.7700002]

l_elbow  x:  215 y:  78
[-0.83204746  0.6264939   2.913     ]

r_wrist  x:  64 y:  81
[0. 0. 0.]

l_wrist  x:  247 y:  55
[-1.0585681  0.7838246  2.352    ]

================frame end ==================
frame : 927
json :  raw_video_000000000927_keypoints.json    /    Pointcloud :  00927.npy
r_shoulder  x:  120 y:  58
[0.54646856 0.8575395  2.6980002 ]

r_elbow  x:  96 y:  94
[0.87562615 0.35696465 2.687     ]

l_shoulder  x:  181 y:  51
[-0.3245404  1.0352589  2.926    ]

l_elbow  x:  216 y:  80
[-0.8275349  0.5828347  2.8460002]

r_wrist  x:  62 y:  90
[0.20874666 0.06396932 0.41700003]

l_wrist  x:  252 y:  62
[-1.0590531  0.6617626  2.226    ]

================frame end ==================
frame : 928
json :  raw_video_00000

l_wrist  x:  249 y:  87
[-0.9409533  0.3450677  2.0440001]

================frame end ==================
frame : 964
json :  raw_video_000000000964_keypoints.json    /    Pointcloud :  00964.npy
r_shoulder  x:  118 y:  58
[0.57185596 0.8540432  2.687     ]

r_elbow  x:  101 y:  103
[0.6519941  0.18809585 2.1720002 ]

l_shoulder  x:  179 y:  52
[-0.  0.  0.]

l_elbow  x:  211 y:  89
[-1.0136569  0.606266   3.8240001]

r_wrist  x:  72 y:  130
[ 1.0893204  -0.12645188  2.4250002 ]

l_wrist  x:  243 y:  97
[-0.  0.  0.]

================frame end ==================
frame : 965
json :  raw_video_000000000965_keypoints.json    /    Pointcloud :  00965.npy
r_shoulder  x:  118 y:  58
[0.57675093 0.8613535  2.71      ]

r_elbow  x:  102 y:  102
[0.62342525 0.19384456 2.1130002 ]

l_shoulder  x:  179 y:  53
[-0.2248267  0.76746    2.2340002]

l_elbow  x:  211 y:  92
[-0.9953665  0.5374389  3.755    ]

r_wrist  x:  79 y:  134
[ 0.9868009  -0.17360754  2.388     ]

l_wrist  x:  241 y:  108
[-1.339

frame : 1004
json :  raw_video_000000001004_keypoints.json    /    Pointcloud :  01004.npy
r_shoulder  x:  112 y:  55
[0.6354536 0.8691389 2.608    ]

r_elbow  x:  107 y:  111
[0.68818706 0.11622836 2.555     ]

l_shoulder  x:  175 y:  51
[-0.  0.  0.]

l_elbow  x:  190 y:  102
[-0.28651094  0.16724023  1.8230001 ]

r_wrist  x:  107 y:  153
[ 0.5295404  -0.33487988  1.9660001 ]

l_wrist  x:  196 y:  140
[-0.33670235 -0.18542635  1.7910001 ]

================frame end ==================
frame : 1005
json :  raw_video_000000001005_keypoints.json    /    Pointcloud :  01005.npy
r_shoulder  x:  112 y:  55
[0.6381338 0.8728048 2.6190002]

r_elbow  x:  107 y:  112
[0.69088054 0.10350244 2.565     ]

l_shoulder  x:  176 y:  51
[-0.  0.  0.]

l_elbow  x:  190 y:  102
[-0.28651094  0.16724023  1.8230001 ]

r_wrist  x:  110 y:  153
[ 0.6112166  -0.40999788  2.407     ]

l_wrist  x:  197 y:  141
[-0.3430088  -0.19299974  1.7760001 ]

================frame end ==================
frame : 1006
json 

cannnot define l_wrist x
================frame end ==================
frame : 1043
json :  raw_video_000000001043_keypoints.json    /    Pointcloud :  01043.npy
r_shoulder  x:  267 y:  12
[-1.1300169  1.1328334  2.0440001]

r_elbow  x:  135 y:  105
[0.33712444 0.20507953 2.687     ]

l_shoulder  x:  267 y:  14
[-1.1300169  1.1118263  2.0440001]

l_elbow  x:  308 y:  80
[-1.0574937   0.28363532  1.3850001 ]

r_wrist  x:  145 y:  169
[ 0.21475157 -0.73215735  2.8990002 ]

cannnot define l_wrist x
================frame end ==================
frame : 1044
json :  raw_video_000000001044_keypoints.json    /    Pointcloud :  01044.npy
r_shoulder  x:  274 y:  12
[-1.122285   1.0563505  1.9060001]

r_elbow  x:  138 y:  104
[0.0673499  0.04985449 0.61200005]

l_shoulder  x:  274 y:  15
[-1.1364167  1.0398985  1.9300001]

l_elbow  x:  313 y:  85
[-0.  0.  0.]

r_wrist  x:  148 y:  167
[ 0. -0.  0.]

cannnot define l_wrist x
================frame end ==================
frame : 1045
json :  raw_vid

{0: [[['r_shoulder', array([-0.04412615, -0.04855199,  0.551     ], dtype=float32)], ['l_shoulder', array([-0.11955939, -0.07792321,  0.60300004], dtype=float32)]]], 1: [[['r_shoulder', array([ 0.03483016, -0.01219401,  0.33200002], dtype=float32)], ['l_shoulder', array([-0.03790483, -0.03668577,  0.54300004], dtype=float32)], ['l_elbow', array([-0.05595508, -0.10613513,  0.55600005], dtype=float32)]]], 2: [[['r_shoulder', array([ 0., -0.,  0.], dtype=float32)], ['l_shoulder', array([-0., -0.,  0.], dtype=float32)], ['l_elbow', array([-0., -0.,  0.], dtype=float32)]]], 3: [[['r_shoulder', array([ 0.07370814, -0.04226642,  0.54300004], dtype=float32)], ['l_shoulder', array([ 0., -0.,  0.], dtype=float32)]]], 4: [[['r_shoulder', array([-0.24871422, -0.2030066 ,  1.636     ], dtype=float32)], ['l_shoulder', array([-1.583091 , -0.5599094,  3.8710003], dtype=float32)]]], 5: [[['r_shoulder', array([-0.22007817, -0.22473985,  1.6110001 ], dtype=float32)], ['l_shoulder', array([-1.4791188 , -0

In [96]:
matched[0]

[[['r_shoulder',
   array([-0.04412615, -0.04855199,  0.551     ], dtype=float32)],
  ['l_shoulder',
   array([-0.11955939, -0.07792321,  0.60300004], dtype=float32)]]]

# 각도 계산하기

In [97]:
def angle(v1, v2, acute):
# v1 is your firsr vector
# v2 is your second vector
    angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    if (acute):
        return 180*(angle /np.pi)
    else:
        return 180*(2 * np.pi - angle)/np.pi

In [98]:
vec1 = [1, -1, 0]
vec2 = [1, 1, 0]

#I am explicitly converting from radian to degree
print(angle(vec1, vec2, True)) #90 degrees
print(angle(vec2, vec1, False)) #270 degrees

90.0
270.0


In [103]:
for ii in matched:
    print('frame :',ii)
    r_shoulder = None
    l_shoulder =None
    r_elbow=None
    l_elbow=None
    r_wrist=None
    l_wrist=None
    
    for points in matched[ii]:
        print(points)
        for point in points:
            if point[0] =='r_shoulder':
                r_shoulder = point[1]
            if point[0] =='l_shoulder':
                l_shoulder = point[1]
            if point[0] =='r_elbow':
                r_elbow = point[1]
            if point[0] =='l_elbow':
                l_elbow = point[1]
            if point[0]=='r_wrist':
                r_wrist=point[1]
            if point[0]=='l_wrist':
                l_wrist == point[1]
    print('success:')
    print(r_shoulder)
    print(l_shoulder)
    print(r_elbow)
    print(l_elbow)
    print(r_wrist)
    print(l_wrist)
    
    try:
        v1 = r_elbow-r_wrist
        v2 = r_elbow-r_shoulder
        print('r_elbow angle : ',angle(v1, v2, False))
    except:
        print('cannot get angle')
        continue
    print('frame end')
    print()

frame : 0
[['r_shoulder', array([-0.04412615, -0.04855199,  0.551     ], dtype=float32)], ['l_shoulder', array([-0.11955939, -0.07792321,  0.60300004], dtype=float32)]]
success:
[-0.04412615 -0.04855199  0.551     ]
[-0.11955939 -0.07792321  0.60300004]
None
None
None
None
cannot get angle
frame : 1
[['r_shoulder', array([ 0.03483016, -0.01219401,  0.33200002], dtype=float32)], ['l_shoulder', array([-0.03790483, -0.03668577,  0.54300004], dtype=float32)], ['l_elbow', array([-0.05595508, -0.10613513,  0.55600005], dtype=float32)]]
success:
[ 0.03483016 -0.01219401  0.33200002]
[-0.03790483 -0.03668577  0.54300004]
None
[-0.05595508 -0.10613513  0.55600005]
None
None
cannot get angle
frame : 2
[['r_shoulder', array([ 0., -0.,  0.], dtype=float32)], ['l_shoulder', array([-0., -0.,  0.], dtype=float32)], ['l_elbow', array([-0., -0.,  0.], dtype=float32)]]
success:
[ 0. -0.  0.]
[-0. -0.  0.]
None
[-0. -0.  0.]
None
None
cannot get angle
frame : 3
[['r_shoulder', array([ 0.07370814, -0.0422

C:\Users\user\Anaconda3\envs\song\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in float_scalars
  after removing the cwd from sys.path.


[['r_shoulder', array([-0.05133254,  0.9613136 ,  2.1790001 ], dtype=float32)], ['r_elbow', array([0.11272512, 0.13054463, 1.4230001 ], dtype=float32)], ['l_shoulder', array([-1.2412972,  1.2450973,  2.7580001], dtype=float32)], ['l_elbow', array([-1.2577243 ,  0.30223095,  2.275     ], dtype=float32)], ['r_wrist', array([ 0.20146005, -0.18479301,  1.4300001 ], dtype=float32)], ['l_wrist', array([-1.4564643 , -0.25891715,  2.275     ], dtype=float32)]]
success:
[-0.05133254  0.9613136   2.1790001 ]
[-1.2412972  1.2450973  2.7580001]
[0.11272512 0.13054463 1.4230001 ]
[-1.2577243   0.30223095  2.275     ]
[ 0.20146005 -0.18479301  1.4300001 ]
None
r_elbow angle :  223.17808767841146
frame end

frame : 184
[['r_shoulder', array([-0.05281669,  0.9775864 ,  2.242     ], dtype=float32)], ['r_elbow', array([0.09679049, 0.13601635, 1.404     ], dtype=float32)], ['l_shoulder', array([-1.183688,  1.173797,  2.63    ], dtype=float32)], ['l_elbow', array([-0.,  0.,  0.], dtype=float32)], ['r_wris

[['r_shoulder', array([0.6067868, 0.9001021, 2.5440001], dtype=float32)], ['r_elbow', array([0.652682  , 0.16315407, 2.292     ], dtype=float32)], ['l_shoulder', array([-0.3059065,  1.0891988,  2.7580001], dtype=float32)], ['l_elbow', array([-0.29083112,  0.16576906,  1.547     ], dtype=float32)], ['r_wrist', array([ 0.9017735 , -0.41054928,  2.565     ], dtype=float32)], ['l_wrist', array([-0.36123052, -0.1484759 ,  1.5090001 ], dtype=float32)]]
success:
[0.6067868 0.9001021 2.5440001]
[-0.3059065  1.0891988  2.7580001]
[0.652682   0.16315407 2.292     ]
[-0.29083112  0.16576906  1.547     ]
[ 0.9017735  -0.41054928  2.565     ]
None
r_elbow angle :  226.65907992853013
frame end

frame : 276
[['r_shoulder', array([0.6198597, 0.9001021, 2.5440001], dtype=float32)], ['r_elbow', array([0.70721734, 0.15831089, 2.397     ], dtype=float32)], ['l_shoulder', array([-0.,  0.,  0.], dtype=float32)], ['l_elbow', array([-0.28932714,  0.1728203 ,  1.539     ], dtype=float32)], ['r_wrist', array([ 

[['r_shoulder', array([0.5014358, 0.9495496, 2.608    ], dtype=float32)], ['r_elbow', array([0.67418087, 0.2553472 , 2.503     ], dtype=float32)], ['l_shoulder', array([-0.44524714,  1.1188182 ,  2.8330002 ], dtype=float32)], ['l_elbow', array([-0.92930484,  0.4738547 ,  3.196     ], dtype=float32)], ['r_wrist', array([ 0.96225846, -0.21042132,  2.388     ], dtype=float32)], ['l_wrist', array([-1.1302869 ,  0.03639641,  2.483     ], dtype=float32)]]
success:
[0.5014358 0.9495496 2.608    ]
[-0.44524714  1.1188182   2.8330002 ]
[0.67418087 0.2553472  2.503     ]
[-0.92930484  0.4738547   3.196     ]
[ 0.96225846 -0.21042132  2.388     ]
None
r_elbow angle :  197.83107808357573
frame end

frame : 369
[['r_shoulder', array([0.46990383, 0.88983864, 2.444     ], dtype=float32)], ['r_elbow', array([0.68253076, 0.25850973, 2.5340002 ], dtype=float32)], ['l_shoulder', array([-0.43927488,  1.103811  ,  2.795     ], dtype=float32)], ['l_elbow', array([-0.8998622,  0.4665005,  3.0410001], dtype=f

...


success:
[0.48263988 0.8701745  2.6520002 ]
[-0.38836518  1.0755252   2.9540002 ]
[0.83954585 0.4276842  2.388     ]
[-0.9011551   0.70653903  2.7580001 ]
[0. 0. 0.]
None
r_elbow angle :  251.14411281992108
frame end

frame : 642
[['r_shoulder', array([0.51103044, 0.9213612 , 2.808     ], dtype=float32)], ['r_elbow', array([0.88349193, 0.45007136, 2.513     ], dtype=float32)], ['l_shoulder', array([-0.38113427,  1.0555001 ,  2.8990002 ], dtype=float32)], ['l_elbow', array([-0.89176416,  0.6883504 ,  2.687     ], dtype=float32)], ['r_wrist', array([0., 0., 0.], dtype=float32)], ['l_wrist', array([-0.8670294,  1.1439726,  2.5340002], dtype=float32)]]
success:
[0.51103044 0.9213612  2.808     ]
[-0.38113427  1.0555001   2.8990002 ]
[0.88349193 0.45007136 2.513     ]
[-0.89176416  0.6883504   2.687     ]
[0. 0. 0.]
None
r_elbow angle :  249.7965301570559
frame end

frame : 643
[['r_shoulder', array([0.499747  , 0.91512865, 2.746     ], dtype=float32)], ['r_elbow', array([0.8873592 , 0.452

[['r_shoulder', array([0.10482676, 0.19195707, 0.57600003], dtype=float32)], ['r_elbow', array([0.17402647, 0.0988278 , 0.49500003], dtype=float32)], ['l_shoulder', array([-0.4334598 ,  0.97581816,  2.7580001 ], dtype=float32)], ['l_elbow', array([-0.92103195,  0.7475236 ,  2.6520002 ], dtype=float32)], ['r_wrist', array([0., 0., 0.], dtype=float32)], ['l_wrist', array([-1.1971195,  1.2568203,  2.7220001], dtype=float32)]]
success:
[0.10482676 0.19195707 0.57600003]
[-0.4334598   0.97581816  2.7580001 ]
[0.17402647 0.0988278  0.49500003]
[-0.92103195  0.7475236   2.6520002 ]
[0. 0. 0.]
None
r_elbow angle :  240.45387639142146
frame end

frame : 735
[['r_shoulder', array([0.49537924, 0.8931429 , 2.7220001 ], dtype=float32)], ['r_elbow', array([0.8560696 , 0.49866572, 2.4350002 ], dtype=float32)], ['l_shoulder', array([-0.46426407,  1.0451657 ,  2.9540002 ], dtype=float32)], ['l_elbow', array([-0.9482878,  0.7611514,  2.6520002], dtype=float32)], ['r_wrist', array([0., 0., 0.], dtype=flo

[-0.4424894  0.2218302  1.889    ]
[ 0.79972434 -0.27537236  2.5340002 ]
None
r_elbow angle :  192.29646651608766
frame end

frame : 825
[['r_shoulder', array([0.5279689, 0.8869068, 2.746    ], dtype=float32)], ['r_elbow', array([0.61942816, 0.19821048, 2.5970001 ], dtype=float32)], ['l_shoulder', array([-0.34806776,  1.016153  ,  2.8720002 ], dtype=float32)], ['l_elbow', array([-0.77560073,  0.40666914,  3.463     ], dtype=float32)], ['r_wrist', array([ 0.7795712 , -0.29556388,  2.5970001 ], dtype=float32)], ['l_wrist', array([-0., -0.,  0.], dtype=float32)]]
success:
[0.5279689 0.8869068 2.746    ]
[-0.34806776  1.016153    2.8720002 ]
[0.61942816 0.19821048 2.5970001 ]
[-0.77560073  0.40666914  3.463     ]
[ 0.7795712  -0.29556388  2.5970001 ]
None
r_elbow angle :  195.91094798334507
frame end

frame : 826
[['r_shoulder', array([0.5230273 , 0.91709566, 2.795     ], dtype=float32)], ['r_elbow', array([0.6325466 , 0.20240824, 2.6520002 ], dtype=float32)], ['l_shoulder', array([-0.3632

[-0.8275349  0.6705835  2.8460002]
[0. 0. 0.]
None
r_elbow angle :  nan
frame end

frame : 917
[['r_shoulder', array([0.55804825, 0.8540432 , 2.687     ], dtype=float32)], ['r_elbow', array([0., 0., 0.], dtype=float32)], ['l_shoulder', array([-0.34031135,  0.9935088 ,  2.808     ], dtype=float32)], ['l_elbow', array([-0.8429458,  0.6830715,  2.8990002], dtype=float32)], ['r_wrist', array([0., 0., 0.], dtype=float32)], ['l_wrist', array([-0.9874692,  1.2139008,  2.8860002], dtype=float32)]]
success:
[0.55804825 0.8540432  2.687     ]
[-0.34031135  0.9935088   2.808     ]
[0. 0. 0.]
[-0.8429458  0.6830715  2.8990002]
[0. 0. 0.]
None
r_elbow angle :  nan
frame end

frame : 918
[['r_shoulder', array([0.56282496, 0.8613535 , 2.71      ], dtype=float32)], ['r_elbow', array([0., 0., 0.], dtype=float32)], ['l_shoulder', array([-0.35303667,  1.0306593 ,  2.913     ], dtype=float32)], ['l_elbow', array([-0.8237549,  0.6675204,  2.8330002], dtype=float32)], ['r_wrist', array([0., 0., 0.], dtype=f


frame : 1012
[['r_shoulder', array([0.51431787, 0.9327056 , 2.6750002 ], dtype=float32)], ['r_elbow', array([0.57563746, 0.10656919, 2.641     ], dtype=float32)], ['l_shoulder', array([-0.4054575,  1.0806224,  2.9680002], dtype=float32)], ['l_elbow', array([-0.7977736 ,  0.29016617,  3.5620003 ], dtype=float32)], ['r_wrist', array([ 0.3017409 , -0.32501647,  1.6580001 ], dtype=float32)], ['l_wrist', array([-0.47179776, -0.207684  ,  1.746     ], dtype=float32)]]
success:
[0.51431787 0.9327056  2.6750002 ]
[-0.4054575  1.0806224  2.9680002]
[0.57563746 0.10656919 2.641     ]
[-0.7977736   0.29016617  3.5620003 ]
[ 0.3017409  -0.32501647  1.6580001 ]
None
r_elbow angle :  246.0313610563297
frame end

frame : 1013
[['r_shoulder', array([0.47605777, 0.92624784, 2.5440001 ], dtype=float32)], ['r_elbow', array([0.56440717, 0.12064095, 2.6520002 ], dtype=float32)], ['l_shoulder', array([-0.41016564,  0.99618214,  2.6980002 ], dtype=float32)], ['l_elbow', array([-0.8538242,  0.3156581,  3.645

array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [11, 12, 13]]])